In [2]:
!python -V

Python 3.9.12


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [4]:
pd.__version__

'1.4.2'

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

In [6]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/07/25 04:47:20 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/25 04:47:20 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-camp/mlruns/1', creation_time=1753216286255, experiment_id='1', last_update_time=1753216286255, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
!pip install pyarrow

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    numerical = ["trip_distance"]

    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [12]:
categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [13]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479562160810692

In [15]:
with open("models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "ivan")
    
    mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

In [16]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [19]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                                                        | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [04:15:36] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.24343                                                                                                         
[1]	validation-rmse:6.71465                                                                                                         
[2]	validation-rmse:6.61205                                                                                                         
[3]	validation-rmse:6.58659                                                                                                         
[4]	validation-rmse:6.57706                                                                                                         
[5]	validation-rmse:6.57063                                                                                                         
[6]	validation-rmse:6.56364                                                                                                         
[7]	validation-rmse:6.55133                                          

[61]	validation-rmse:6.48850                                                                                                        
[62]	validation-rmse:6.48856                                                                                                        
[63]	validation-rmse:6.48858                                                                                                        
[64]	validation-rmse:6.48754                                                                                                        
[65]	validation-rmse:6.48727                                                                                                        
[66]	validation-rmse:6.48751                                                                                                        
[67]	validation-rmse:6.48765                                                                                                        
[68]	validation-rmse:6.48750                                         

[122]	validation-rmse:6.48034                                                                                                       
[123]	validation-rmse:6.48027                                                                                                       
[124]	validation-rmse:6.47968                                                                                                       
[125]	validation-rmse:6.48022                                                                                                       
[126]	validation-rmse:6.48020                                                                                                       
[127]	validation-rmse:6.47947                                                                                                       
[128]	validation-rmse:6.47968                                                                                                       
[129]	validation-rmse:6.48030                                        

...

[488]	validation-rmse:6.61157                                                                                                       
[489]	validation-rmse:6.61149                                                                                                       
[490]	validation-rmse:6.61148                                                                                                       
[491]	validation-rmse:6.61146                                                                                                       
[492]	validation-rmse:6.61145                                                                                                       
[493]	validation-rmse:6.61135                                                                                                       
[494]	validation-rmse:6.61132                                                                                                       
[495]	validation-rmse:6.61122                                        

[549]	validation-rmse:6.60939                                                                                                       
[550]	validation-rmse:6.60932                                                                                                       
[551]	validation-rmse:6.60929                                                                                                       
[552]	validation-rmse:6.60928                                                                                                       
[553]	validation-rmse:6.60926                                                                                                       
[554]	validation-rmse:6.60926                                                                                                       
[555]	validation-rmse:6.60923                                                                                                       
[556]	validation-rmse:6.60921                                        

[610]	validation-rmse:6.60728                                                                                                       
[611]	validation-rmse:6.60725                                                                                                       
[612]	validation-rmse:6.60706                                                                                                       
[613]	validation-rmse:6.60704                                                                                                       
[614]	validation-rmse:6.60702                                                                                                       
[615]	validation-rmse:6.60697                                                                                                       
[616]	validation-rmse:6.60696                                                                                                       
[617]	validation-rmse:6.60686                                        

[671]	validation-rmse:6.60439                                                                                                       
[672]	validation-rmse:6.60433                                                                                                       
[673]	validation-rmse:6.60429                                                                                                       
[674]	validation-rmse:6.60423                                                                                                       
[675]	validation-rmse:6.60414                                                                                                       
[676]	validation-rmse:6.60411                                                                                                       
[677]	validation-rmse:6.60410                                                                                                       
[678]	validation-rmse:6.60414                                        

KeyboardInterrupt: 

In [18]:
params = {"learning_rate" :0.06577369757142525,
"max_depth": 52,
"min_child_weight": 1.305048390995635,
"objective": "reg:linear",
"reg_alph": 0.07572303274527586,
"reg_lambda": 0.11450931697791678,
"seed": 42}

In [19]:
mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)

2025/07/25 04:47:39 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c338048e0fe9487389fba15cefac4427', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [04:47:43] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [04:47:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "reg_alph" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.67729
[1]	validation-rmse:11.18704
[2]	validation-rmse:10.73918
[3]	validation-rmse:10.33010
[4]	validation-rmse:9.95809
[5]	validation-rmse:9.61997
[6]	validation-rmse:9.31264
[7]	validation-rmse:9.03290
[8]	validation-rmse:8.78029
[9]	validation-rmse:8.55257
[10]	validation-rmse:8.34703
[11]	validation-rmse:8.16157
[12]	validation-rmse:7.99584
[13]	validation-rmse:7.84511
[14]	validation-rmse:7.71088
[15]	validation-rmse:7.58931
[16]	validation-rmse:7.48081
[17]	validation-rmse:7.38253
[18]	validation-rmse:7.29537
[19]	validation-rmse:7.21573
[20]	validation-rmse:7.14559
[21]	validation-rmse:7.08248
[22]	validation-rmse:7.02482
[23]	validation-rmse:6.97292
[24]	validation-rmse:6.92647
[25]	validation-rmse:6.88573
[26]	validation-rmse:6.84793
[27]	validation-rmse:6.81431
[28]	validation-rmse:6.78365
[29]	validation-rmse:6.75572
[30]	validation-rmse:6.73111
[31]	validation-rmse:6.70836
[32]	validation-rmse:6.68771
[33]	validation-rmse:6.66894
[34]	validation-rmse

[277]	validation-rmse:6.37377
[278]	validation-rmse:6.37353
[279]	validation-rmse:6.37325
[280]	validation-rmse:6.37307
[281]	validation-rmse:6.37290
[282]	validation-rmse:6.37277
[283]	validation-rmse:6.37254
[284]	validation-rmse:6.37217
[285]	validation-rmse:6.37195
[286]	validation-rmse:6.37170
[287]	validation-rmse:6.37159
[288]	validation-rmse:6.37120
[289]	validation-rmse:6.37096
[290]	validation-rmse:6.37069
[291]	validation-rmse:6.37057
[292]	validation-rmse:6.37041
[293]	validation-rmse:6.37019
[294]	validation-rmse:6.36996
[295]	validation-rmse:6.36978
[296]	validation-rmse:6.36959
[297]	validation-rmse:6.36960
[298]	validation-rmse:6.36951
[299]	validation-rmse:6.36925
[300]	validation-rmse:6.36916
[301]	validation-rmse:6.36896
[302]	validation-rmse:6.36881
[303]	validation-rmse:6.36866
[304]	validation-rmse:6.36850
[305]	validation-rmse:6.36833
[306]	validation-rmse:6.36811
[307]	validation-rmse:6.36790
[308]	validation-rmse:6.36777
[309]	validation-rmse:6.36758
[310]	vali

[551]	validation-rmse:6.33448
[552]	validation-rmse:6.33439
[553]	validation-rmse:6.33434
[554]	validation-rmse:6.33425
[555]	validation-rmse:6.33413
[556]	validation-rmse:6.33397
[557]	validation-rmse:6.33397
[558]	validation-rmse:6.33389
[559]	validation-rmse:6.33378
[560]	validation-rmse:6.33379
[561]	validation-rmse:6.33379
[562]	validation-rmse:6.33368
[563]	validation-rmse:6.33360
[564]	validation-rmse:6.33358
[565]	validation-rmse:6.33351
[566]	validation-rmse:6.33337
[567]	validation-rmse:6.33326
[568]	validation-rmse:6.33312
[569]	validation-rmse:6.33288
[570]	validation-rmse:6.33267
[571]	validation-rmse:6.33266
[572]	validation-rmse:6.33263
[573]	validation-rmse:6.33249
[574]	validation-rmse:6.33246
[575]	validation-rmse:6.33245
[576]	validation-rmse:6.33242
[577]	validation-rmse:6.33243
[578]	validation-rmse:6.33225
[579]	validation-rmse:6.33219
[580]	validation-rmse:6.33215
[581]	validation-rmse:6.33205
[582]	validation-rmse:6.33196
[583]	validation-rmse:6.33190
[584]	vali

[825]	validation-rmse:6.31382
[826]	validation-rmse:6.31374
[827]	validation-rmse:6.31357
[828]	validation-rmse:6.31340
[829]	validation-rmse:6.31329
[830]	validation-rmse:6.31327
[831]	validation-rmse:6.31318
[832]	validation-rmse:6.31318
[833]	validation-rmse:6.31311
[834]	validation-rmse:6.31305
[835]	validation-rmse:6.31304
[836]	validation-rmse:6.31299
[837]	validation-rmse:6.31302
[838]	validation-rmse:6.31304
[839]	validation-rmse:6.31304
[840]	validation-rmse:6.31294
[841]	validation-rmse:6.31284
[842]	validation-rmse:6.31274
[843]	validation-rmse:6.31270
[844]	validation-rmse:6.31268
[845]	validation-rmse:6.31269
[846]	validation-rmse:6.31270
[847]	validation-rmse:6.31276
[848]	validation-rmse:6.31255
[849]	validation-rmse:6.31245
[850]	validation-rmse:6.31242
[851]	validation-rmse:6.31234
[852]	validation-rmse:6.31219
[853]	validation-rmse:6.31223
[854]	validation-rmse:6.31224
[855]	validation-rmse:6.31221
[856]	validation-rmse:6.31218
[857]	validation-rmse:6.31218
[858]	vali

2025/07/25 04:52:22 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/07/25 04:52:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/25 04:52:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [04:52:23] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/07/25 04:52:31 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpaqhcmdsc/model, flavor: xgboost). Fall back to return ['xgboost==2.1.4']. Set logging level to DEBUG to see the full traceback. 
2025/07/25 04:52:31 WARNING mlflow.models.mod